# HW4
## Afek Adler
January 7, 2020


# Config

In [1]:
DATASET = 2
WRITE_TO_CSV = False
ATT_RESULTS_PATH_DATA1 = "ATT_results1.csv"
ATT_RESULTS_PATH_DATA2 = "ATT_results2.csv"
ATT_RESULTS_PATH_DATA = ATT_RESULTS_PATH_DATA1 if DATASET ==1 else ATT_RESULTS_PATH_DATA2
PROPENSITY_SCORES_PATH = "models_propensity.csv"
SEED = 1

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
np.random.seed(SEED)

## IO

In [3]:
if DATASET == 1:
    df = pd.read_csv('data1.csv',index_col = 0)
else:
    df = pd.read_csv('data2.csv',index_col = 0)
display(df.head(2))

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_51,x_52,x_53,x_54,x_55,x_56,x_57,x_58,T,Y
1,29,C,1.0,7.0,60,85,0,0,1,0,...,0,0,0,0,0,0,45,39,1,26.786273
2,27,C,0.0,0.0,64,178,0,0,0,0,...,0,0,0,0,0,0,46,42,1,0.854251


## Preprocessing 
* We know that there are no missing values

#### 1. Discard feutres which containg a unique value more than 95% of the time

In [4]:
from scipy.stats import entropy
# Discard feutres which containg a unique value more than 95% of the time / low entropy
low_entropy_columns = []
for col in df.columns:
    series = df[col].value_counts()/df.shape[0]
    humogenious = entropy(series.values)
    if humogenious <= 0.3:
        low_entropy_columns.append(col)

print("Those are indedd features that tend to get a specific value \nAnd this specific value is present more than 95% of the time:")
for col in low_entropy_columns:
    series = df[col].value_counts()
    print(series.iloc[0]/df.shape[0])
    
columns = [col for col in df.columns if col not in low_entropy_columns]
df = df[columns]

Those are indedd features that tend to get a specific value 
And this specific value is present more than 95% of the time:
0.9852144939608496
0.9775093710953769
0.9966680549770929
0.9735526863806747
0.9473136193252811
0.9808413161182841
0.9716784673052895
0.9777176176593086
0.9968763015410246
0.9593919200333194
0.9700124947938359
0.9975010412328197
0.994585589337776


#### 2. Treat catrgorical variables - I would choose a method that will be valid for all the methods below. So for example encoding categories by their mean response value is not a good solution, because one time we predict T and the other Y.

In [5]:
categorical_dtypes = df[columns].select_dtypes(include=['object'])
for col in categorical_dtypes:
    print(col)
    display(categorical_dtypes[col].value_counts())

x_21


J    2561
I     875
L     339
F     221
H     215
K     147
B      87
O      73
G      72
A      59
N      49
D      45
P      25
C      16
E      10
M       8
Name: x_21, dtype: int64

x_24


E    3449
B    1273
D      52
A      18
C      10
Name: x_24, dtype: int64

In [6]:
x_21_values_counts = categorical_dtypes['x_21'].value_counts().to_dict()
x_21_map = {i: (i  if v > 100 else 'other') for i,v in x_21_values_counts.items()}
x_21_map

{'J': 'J',
 'I': 'I',
 'L': 'L',
 'F': 'F',
 'H': 'H',
 'K': 'K',
 'B': 'other',
 'O': 'other',
 'G': 'other',
 'A': 'other',
 'N': 'other',
 'D': 'other',
 'P': 'other',
 'C': 'other',
 'E': 'other',
 'M': 'other'}

In [7]:
# make x_24 binary and x_21 to one hot encoding 
df['x_24'] = (df['x_24'] == 'E')*1
df = pd.concat([pd.get_dummies(df['x_21'].map(x_21_map),prefix = 'x_21_'), df.drop(columns = 'x_21')], axis=1)

# Models

In [8]:
class EstimatorSelectionHelper:
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        self.best_model = None

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=True):
        best_score = 0
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X, y)
            if gs.best_score_ > best_score:
                best_score = gs.best_score_
                self.best_model = gs.best_estimator_
            self.grid_searches[key] = gs
        return self.best_model

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})
        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))
            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))      
        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns]

## 1. Inverse propensity score weighting
(From Abdia)  

$\hat{\mu}_{A T T, I P W}=\frac{\sum_{i=1}^{n} Z_{i} Y_{i}}{\sum_{i=1}^{n} Z_{i}}-\frac{\sum_{i=1}^{n}\left(1-Z_{i}\right) Y_{i} e\left(X_{i}\right) /\left(1-e\left(X_{i}\right)\right)}{\sum_{i=1}^{n}\left(1-Z_{i}\right) e\left(X_{i}\right) /\left(1-e\left(X_{i}\right)\right)}$  
WE need to estimate $e(x_i) = p(x_i = 1)$

In [9]:
x_cols = [col for col in df.columns if col.startswith('x')]
X = df[x_cols]
Y = df['T']
atts = {}

In [10]:
# naive solution
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
models = {
    'lr': LogisticRegression(),'rf': RandomForestClassifier(),'mlp':MLPClassifier()}
params = {
    'lr': [{'solver': ['lbfgs'],'C': [2], 'class_weight':['balanced']},
           {'solver': ['lbfgs'],'C': [0.1,0.2,1], 'class_weight':['balanced']} ],
    'rf':[{'n_estimators' : [100], 'max_depth':[3,4,5,None]}],
    'mlp': [{'hidden_layer_sizes': [(10,5)]}]}

helper = EstimatorSelectionHelper(models, params)
best_model = helper.fit(X, Y, scoring='roc_auc', n_jobs=8, cv =5)
helper.score_summary(sort_by='max_score')
print('best model : ')
print (best_model)

Running GridSearchCV for lr.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    2.8s finished
C:\Users\afeki\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for rf.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    1.7s finished


Running GridSearchCV for mlp.
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:    0.9s remaining:    1.4s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:    1.4s finished


best model : 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


C:\Users\afeki\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [11]:
"""
We can look at the train vs test score of our best model in order to see how much we are over (under) fitting.
In fact, anyhow, i will not further fine tune the best model.
"""
pd.DataFrame(helper.grid_searches['rf'].cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.311875,0.022912,0.020346,0.001954,3,100,"{'max_depth': 3, 'n_estimators': 100}",0.739134,0.703729,0.767387,...,0.725731,0.027182,4,0.742774,0.754634,0.744282,0.747800,0.754359,0.748770,0.004953
1,0.349371,0.019282,0.021144,0.000399,4,100,"{'max_depth': 4, 'n_estimators': 100}",0.736539,0.705677,0.767943,...,0.727365,0.024266,2,0.766111,0.776815,0.765843,0.768865,0.778117,0.771150,0.005280
2,0.384371,0.008776,0.020146,0.000977,5,100,"{'max_depth': 5, 'n_estimators': 100}",0.748243,0.710835,0.772256,...,0.732126,0.025365,1,0.800980,0.809594,0.799372,0.801298,0.810391,0.804327,0.004679
3,0.778515,0.057882,0.022739,0.002475,None,100,"{'max_depth': None, 'n_estimators': 100}",0.746324,0.705817,0.765099,...,0.726216,0.025207,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [12]:
# ATT results
v = best_model.predict_proba(X)[:,1]/best_model.predict_proba(X)[:,0]
ATT = (df['T']*df['Y']).sum()/(df['T'].sum()) - ((1-df['T'])*df['Y']*v).sum()/((1-df['T'])*v).sum()
propensity_scores = pd.DataFrame(best_model.predict_proba(X)[:,1]).T
propensity_scores.index = ['data1' if DATASET == 1 else 'data2']
atts['1'] = ATT
ATT

3.5581220732948005

# 2. S Learner
$\hat{y} = f(x,t) \rightarrow$  
$CATE(x) = f(x,1) - f(x,0) \rightarrow$  
$ATT = \sum_{i\
in \{T =1\}}f(x,1) - f(x,0)$  
We need to fit a model $\hat{y} = f(x,t) $

In [13]:
X = df[x_cols+['T']]
Y = df['Y']

In [14]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
models = {
    'lr':  Ridge(),'rf': RandomForestRegressor()}
params = {
    'lr': [{'alpha':[0.1,0.5,1]}],
    'rf':[{'n_estimators' : [100], 'max_depth':[3,4,5,None]}]}

helper = EstimatorSelectionHelper(models, params)
best_model = helper.fit(X, Y, n_jobs=8, cv =5)
helper.score_summary(sort_by='max_score')
print('best model : ')
print (best_model)

Running GridSearchCV for lr.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Running GridSearchCV for rf.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    8.7s finished


best model : 
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


C:\Users\afeki\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [15]:
pd.DataFrame(helper.grid_searches['rf'].cv_results_)[['params','mean_train_score','mean_test_score']]

,params,mean_train_score,mean_test_score
0,"{'max_depth': 3, 'n_estimators': 100}",0.800755,0.776987
1,"{'max_depth': 4, 'n_estimators': 100}",0.871686,0.850353
2,"{'max_depth': 5, 'n_estimators': 100}",0.909522,0.888914
3,"{'max_depth': None, 'n_estimators': 100}",0.990618,0.930802


In [16]:
x1 = X[X['T'] == 1]
x0 = x1.copy()
x0['T'] = 0
ATT = np.mean(best_model.predict(x1)- best_model.predict(x0))
atts['2'] = ATT
ATT

2.212959612199516

# 3. T Learner
model #1 $\hat{y_1} = f(x,t = 1) = f_1$  
model #2 $\hat{y_0} = f(x,t = 0) = f_0$

$CATE(x) = f_1(x) - f_0(x) \rightarrow$  
$ATT = \sum_{i\in \{T=1\}}f_1(x) - f_0(x)$  
We need to fit 2 models $f_0, f_1$

In [17]:
X_0 = df[df['T'] == 0][x_cols]
Y_0 = df[df['T'] == 0]['Y']
X_1 = df[df['T'] == 1][x_cols]
Y_1 = df[df['T'] == 1]['Y']

In [18]:
from sklearn.linear_model import Ridge
models = {
    'lr':  Ridge(),'rf': RandomForestRegressor()}
params = {
    'lr': [{'alpha':[0.1,0.5,1]}],
    'rf':[{'n_estimators' : [100], 'max_depth':[3,4,5,None]}]}

helper = EstimatorSelectionHelper(models, params)
best_model = helper.fit(X_0, Y_0, n_jobs=8, cv =5)
helper.score_summary(sort_by='max_score')
print('best model : ')
print (best_model)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for lr.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Running GridSearchCV for rf.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    3.9s finished


best model : 
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


C:\Users\afeki\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [19]:
best_model1 = helper.fit(X_1, Y_1, n_jobs=8, cv =5)
helper.score_summary(
    sort_by='max_score')
print('best model : ')
print (best_model)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Running GridSearchCV for lr.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Running GridSearchCV for rf.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    4.4s finished


best model : 
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


C:\Users\afeki\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [20]:
ATT = np.mean(best_model1.predict(X_1)- best_model.predict(X_1))
atts['3'] = ATT

# 4. Matching
For each $x_i \in \{T = 1\}$  estimate $y_0^i $ based on $x_i$ nearest neighbors 

In [21]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_0,Y_0)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [22]:
ATT = np.mean(Y_1.values - knn.predict(X_1)) 
atts['4'] = ATT
ATT

3.371270606026528

In [23]:
if WRITE_TO_CSV:
    propensity_scores.to_csv(PROPENSITY_SCORES_PATH, mode = 'a',header = None)
    pd.Series(atts).to_frame().to_csv(ATT_RESULTS_PATH_DATA,header = None, index = None)